### Downloading the Dogs vs Cats dataset


In [ ]:
 !wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2021-06-27 05:50:24--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.195.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   157MB/s    in 0.4s    

2021-06-27 05:50:24 (157 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Dataset preprocessing


In [ ]:
import os ## it helps us to work with folders,locate files and paths.
import zipfile ## it helps to extract zip files
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook ## helps us to visualize any progress in the project
from tensorflow.keras.preprocessing.image import ImageDataGenerator ## it helps us to create automatic Image preprocessing pipeline

%matplotlib inline
tf.__version__

'2.5.0'

In [ ]:
dataset_path = './cats_and_dogs_filtered.zip'
zip_object = zipfile.ZipFile(dataset_path, mode = 'r')

In [ ]:
zip_object.extractall('./')

In [ ]:
zip_object.close()

In [ ]:
## to use ImageDataGenerator we need to have all the classes in different folders

In [ ]:
dataset_path_new = './cats_and_dogs_filtered/'
train_dr = os.path.join(dataset_path_new,'train')
validation_dr = os.path.join(dataset_path_new, 'validation')

In [ ]:
train_dr

'./cats_and_dogs_filtered/train'

## Building the model

In [ ]:
## Loading the pre-traine model(MobileNetV2)
IMG_SHAPE = (128,128 , 3)

####Loading MobileNetV2 - Pre-trained model

In [ ]:
## many models are there in keras, input shape is fixed for any CNN, Include top should have to be False as this model is trained in the imagenet dataset so we want to change the head of the model (the final layer) so that we can customize the model for our purpose
## weights that we want are the weights of it by training in the IMageNet dataset.
base_model = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE, include_top = False, weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
## To check the model Artchiture
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [ ]:
## Freezing the base Model, otherwise the initial weights will change and we only want to change the weights of the 
base_model.trainable = False

In [ ]:
## Defining the custom head for our network
base_model.output
## This size is not very suitable for the output layer of the custom head

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [ ]:
## There are some approaches to handle this problem
## 1.we could use the Flattening layer to convert or reshape the output to vectors, but here we have big vectors for our custom part of our network.
## 2. here the better solution is to use Global average Pooling layer(similar to the MaxPooling layer)
##    Global word means that it's gona take the whole input instead of processing parts of it at a time and this feature will help us. it reduces the input size significantly
##    Average - instead of finding the most significant value( what the Max pooling is doing), this layer takes an average of all numbers in the input.

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [ ]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [ ]:
## defining the output layer / prediction layer. and input to this is global_average_layer.
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

In [ ]:
prediction_layer

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense')>

In [ ]:
## Now we are going to create final model by combining the base model with our custom head.
## uptil now we created sequential models but in this case since we are combining two neural networks we are going to use a simple model which allows us to specify the inputs and outputs separately 
model = tf.keras.models.Model(inputs = base_model.input, outputs = prediction_layer)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [ ]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr = 0.0001), loss = 'binary_crossentropy', metrics = ['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
## Creating Data Generators
## Resizing images
data_gen_train = ImageDataGenerator(rescale = 1/255.0)
data_gen_valid = ImageDataGenerator(rescale = 1/255.0)

In [ ]:
train_generator = data_gen_train.flow_from_directory(train_dr, target_size = (128,128), batch_size = 128, class_mode = 'binary')

Found 2000 images belonging to 2 classes.


In [ ]:
valid_generator = data_gen_valid.flow_from_directory(validation_dr, target_size = (128,128), batch_size = 128, class_mode = 'binary')

Found 1000 images belonging to 2 classes.


In [ ]:
train_generator

### Training the model

In [ ]:
# Training the Transfer learning model
# when using a data generator for preprocessing the dataset, we should use a fit generator function instead of normal fit function on our model.
model.fit(train_generator, epochs = 10, validation_data= valid_generator)

Epoch 1/10
16/16 [==============================] - 41s 559ms/step - loss: 0.5289 - accuracy: 0.7515 - val_loss: 0.5019 - val_accuracy: 0.7790
Epoch 2/10
16/16 [==============================] - 7s 467ms/step - loss: 0.4470 - accuracy: 0.8205 - val_loss: 0.4379 - val_accuracy: 0.8200
Epoch 3/10
16/16 [==============================] - 7s 470ms/step - loss: 0.3913 - accuracy: 0.8625 - val_loss: 0.3878 - val_accuracy: 0.8550
Epoch 4/10
16/16 [==============================] - 7s 470ms/step - loss: 0.3461 - accuracy: 0.8840 - val_loss: 0.3467 - val_accuracy: 0.8820
Epoch 5/10
16/16 [==============================] - 8s 483ms/step - loss: 0.3089 - accuracy: 0.9010 - val_loss: 0.3123 - val_accuracy: 0.8940
Epoch 6/10
16/16 [==============================] - 7s 470ms/step - loss: 0.2785 - accuracy: 0.9125 - val_loss: 0.2844 - val_accuracy: 0.9060
Epoch 7/10
16/16 [==============================] - 7s 474ms/step - loss: 0.2538 - accuracy: 0.9200 - val_loss: 0.2608 - val_accuracy: 0.9130
Epoch

### Transfer learning model evaluation

In [ ]:
# Transfer learning model evaluation
valid_loss, valid_accuracy = model.evaluate(valid_generator)

8/8 [==============================] - 3s 307ms/step - loss: 0.2082 - accuracy: 0.9280


In [ ]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9279999732971191



## Fine Tuning





In [ ]:
## Use it after Transfer Learning, to improve the results.
## In transfer Learning we used a pre trained network that we have frozen and just trained the custom part of it to be suited for the task that we're trying to solve.
## but the base network at this point it doesn't have any features connected to our specific task, so if we have very specific task to solve, it is good to have features from the data set inside the base network as well.  

In [ ]:
# we unfreeze a few top layers from the base network, this way we allow the network to learn even further about the custom data set.
##  Don't use fine tuning on the whole network , only a few top layers are enough, the reason is that in most cases they are more specialized for the specific task. When CNN is trained, lower layers are going to learn general features like edges corners and all, whereas the top layers are specific to the dataset( like eyes, ears , tails etc)
##  Use it after Transfer Learning, to improve the results. reason is that if we try to perform fine tuning immediately in our custom head which is not trained, graidents will be much different between our custom head layer and a few unfrozen layers from the base model. 

In [ ]:
# unfreezing the top layers
base_model.trainable = True

In [ ]:
print('Number of layers in the base model: {}'.format(len(base_model.layers)))

Number of layers in the base model: 154


In [ ]:
base_model.layers

In [ ]:
fine_tune_at = 100

In [ ]:
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr = 0.0001), loss = 'binary_crossentropy', metrics = ['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
model.fit(train_generator, epochs= 5, validation_data = valid_generator)

Epoch 1/5
16/16 [==============================] - 14s 523ms/step - loss: 0.1566 - accuracy: 0.9400 - val_loss: 0.0798 - val_accuracy: 0.9660
Epoch 2/5
16/16 [==============================] - 8s 475ms/step - loss: 0.0211 - accuracy: 0.9980 - val_loss: 0.0835 - val_accuracy: 0.9660
Epoch 3/5
16/16 [==============================] - 8s 478ms/step - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.0893 - val_accuracy: 0.9670
Epoch 4/5
16/16 [==============================] - 8s 480ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.0897 - val_accuracy: 0.9650
Epoch 5/5
16/16 [==============================] - 8s 476ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.1051 - val_accuracy: 0.9670


In [ ]:
valid_loss, valid_accuracy = model.evaluate(valid_generator)

8/8 [==============================] - 3s 310ms/step - loss: 0.1051 - accuracy: 0.9670


In [ ]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9670000076293945
